# 7.21 Gases at Sonic Velocity

Coke oven gas has a specific gravity of 0.42, a header pressure of 125 psig, and is at 140 deg F. It flows through 20 feet of 3" schedule 40 pipe and is discharged to the atmosphere.

Find the flow rate in standard cubic feet per hour.

In [1]:
from fluids.units import *
from math import pi
SG = 0.42
P1 = 125*u.psi + 1.0*u.atmosphere
P2 = 1.0*u.atmosphere
roughness = 0.0018*u.inch

L = 20*u.foot
mu = 1.8e-5 # good guess

NPS, Di, Do, t = nearest_pipe(NPS=3, schedule='40')
A = 0.25*pi*Di**2

fd = 0.0175 # initial guess
P_choke = P_isothermal_critical_flow(P=P1, fd=fd, L=L, D=Di)*(1+1e-10)
P_rat_rho = (0.5*(P1 + P_choke)/(1.0*u.atmosphere)).to_base_units()

for i in range(5):
    rho = 1.2*u.kg/u.m**3*SG*P_rat_rho
    m = isothermal_gas(rho, fd, P1=P1, P2=P_choke, L=L, D=Di)
    Q_actual = (m/rho).to(u.ft**3/u.hour)
    Q_std = Q_actual*P_rat_rho # convert to SI
    v = Q_actual/A
    Re = rho*v*Di/mu
    fd = friction_factor(Re=Re, eD=roughness/Di)
    K = exit_normal() + entrance_sharp(method='Crane')
    
    # Increase fd so it accounts for all the extra losses
    fd += K*Di/L
    P_choke = P_isothermal_critical_flow(P=P1, fd=fd, L=L, D=Di)*(1+1e-10)
    P_rat_rho = (0.5*(P1 + P_choke)/(1.0*u.atmosphere)).to_base_units()
    print(Q_std)

1176795.8398551396 foot ** 3 / hour
922482.9941880251 foot ** 3 / hour
922210.5721001471 foot ** 3 / hour
922210.2011294398 foot ** 3 / hour
922210.2006241237 foot ** 3 / hour


The result given in Crane is 1028000 ft^3/hour, which is 10% higher than this result.
The crane result assumes isentropic flow whereas this is for isothermal flow. 

THe most accurate answer that can be obtained for this case would be quite a bit better than either solution here. A thermodynamic calculation of the direct conditions of choke:
    
    speed of sound of the fluid is the speed of the fluid
    energy is maintained
However, without the composition of the gas there is no point in trying to obtain a better result.